In [159]:
import numpy as np
import copy

In [ ]:
# gram-schmidt orthogonalisation algorithm
def gs_algorith(B):

    d = len(B)
    B_star = copy.deepcopy(B)
    mu = np.zeros((d, d))
    
    for i in range(1, d):
        bi_star = B_star[i]
        
        for j in range(i):
            # project b_i onto b^*_j
            mu[i, j] = np.dot(B[i], B_star[j]) / np.dot(B_star[j], B_star[j])
            bi_star = bi_star - mu[i, j] * B_star[j]
            
        B_star[i] = bi_star
        
    return B_star, mu
        

# size reduction algorithm
def size_reduction(B):

    B_reduced = copy.deepcopy(B)
    d = len(B_reduced)
    # pre-compute the GS coefficients for efficiency
    mu = gs_algorith(B_reduced)[1]

    # overwrite GS coefficient \mu_{ii} = 1 to make later update correct
    for j in range(d):
        mu[j, j] = 1

    for i in range(1, d):
        for j in range(i - 1, -1, -1):
            B_reduced[i] = B_reduced[i] - round(mu[i, j]) * B_reduced[j]

            # update corresponding GS coefficients due to the reduction of b_i
            for k in range(j + 1):
                mu[i, k] = mu[i, k] - round(mu[i, j]) * mu[j, k]
                
    return B_reduced
    

# LLL reduction algorithm
def lll_reduction(B, delta = 3/4):

    B_reduced = copy.deepcopy(B)
    d = len(B_reduced)
    
    while True:
        # track whether any swap happened
        changed = False 
        
        B_reduced = size_reduction(B_reduced)
    
        B_star, mu = gs_algorith(B_reduced)
        
        for i in range(d - 1):
            # check lovasz condition
            lhs = np.inner(B_star[i + 1], B_star[i + 1])
            rhs = (delta - mu[i + 1, i] ** 2) * np.inner(B_star[i], B_star[i])
            if lhs < rhs:
                # print("Lovasz condition not met, swap {i} and {i + 1}")
                
                # swap row i and i + 1
                B_reduced[i], B_reduced[i + 1] = B_reduced[i + 1].copy(), B_reduced[i].copy()

                changed = True
                # exit LLL loop to restart
                break
        
        if not changed:
            return B_reduced


In [188]:
B = [np.array([92, 44, 95, 5, 97]), 
     np.array([58, 43, 99, 37, 68]), 
     np.array([26, 95, 16, 89, 33]), 
     np.array([17, 51, 55, 42, 82]), 
     np.array([24, 89, 92, 59, 92])]

B = np.array(B)

B

array([[92, 44, 95,  5, 97],
       [58, 43, 99, 37, 68],
       [26, 95, 16, 89, 33],
       [17, 51, 55, 42, 82],
       [24, 89, 92, 59, 92]])

In [193]:
print(np.round(np.linalg.norm(B, axis = 1), 2))
print(np.round(gs_algorith(B)[1], 2))

[169.88 144.94 137.72 120.01 170.02]
[[0.   0.   0.   0.   0.  ]
 [0.81 0.   0.   0.   0.  ]
 [0.41 1.66 0.   0.   0.  ]
 [0.6  0.94 0.3  0.   0.  ]
 [0.83 1.68 0.42 1.01 0.  ]]


In [137]:
B_reduced = size_reduction(B)
B_reduced

array([[ 92,  44,  95,   5,  97],
       [-34,  -1,   4,  32, -29],
       [  2,  53, -87,  20,  -6],
       [-41,   8, -44,   5,  14],
       [ 41,  39,  33, -15,  39]])

In [194]:
print(np.round(np.linalg.norm(B_reduced, axis = 1), 2))
print(np.round(gs_algorith(B_reduced)[1], 2))

[169.88  55.12 104.01  62.47  77.7 ]
[[ 0.    0.    0.    0.    0.  ]
 [-0.19  0.    0.    0.    0.  ]
 [-0.22 -0.42  0.    0.    0.  ]
 [-0.22 -0.1   0.3   0.    0.  ]
 [ 0.43 -0.3   0.13 -0.07  0.  ]]


In [179]:
B_lll = lll_reduction(B)
B_lll

array([[-34,  -1,   4,  32, -29],
       [-41,   8, -44,   5,  14],
       [  7,  38,  37,  17,  10],
       [ 10, -34,  29,  35,  19],
       [ 53,  11, -14,  50,  -1]])

In [195]:
print(np.round(np.linalg.norm(B_lll, axis = 1), 2))
print(np.round(gs_algorith(B_lll)[1], 2))

[55.12 62.47 57.02 60.69 75.01]
[[ 0.    0.    0.    0.    0.  ]
 [ 0.32  0.    0.    0.    0.  ]
 [ 0.04 -0.4   0.    0.    0.  ]
 [ 0.12 -0.46 -0.03  0.    0.  ]
 [-0.08 -0.32  0.26  0.35  0.  ]]
